In [ ]:
import pandas as pd
#use to connect database
import sqlalchemy as sal
engine=sal.create_engine('mssql://LAPTOP-27\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [ ]:
def extract():
    #read text file
    df_product=pd.read_csv('products.txt')
    #read sql table
    df_product_db=pd.read_sql_query("select * from product_dim where end_date = '9999-12-31' " ,conn)
    return df_product,df_product_db
    
def transform(df_product,df_product_db):
    df_merged=pd.merge(df_product,df_product_db,how='inner',left_on='product_id',right_on='product_id')
    update_rows= df_merged['product_key']
    keys = update_rows.to_list()
    product_keys= ','.join([str(key) for key in keys])
    return product_keys
    
def inserts(df_product):
    df_product['start_date'] =pd.to_datetime('now').strftime('%Y-%m-%d')
    df_product['end_date'] = '9999-12-31'
    df_product.to_sql('product_dim',con=conn , index=False , if_exists = 'append')
    conn.commit()
    
def updates(product_keys):
    query = sal.text("update product_dim set end_date =  cast(getdate()-1 as date) where product_key in (" + product_keys + ")")
    p = conn.execute(query)
    conn.commit()    

In [ ]:
df_product,df_product_db=extract()
product_keys=transform(df_product,df_product_db)
inserts(df_product)
if product_keys!='':
    updates(product_keys)